In [6]:
# ✅ Step 1: Import Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, Flatten, Dense, Dropout

# ✅ Step 2: Load Preprocessed Data
print("📥 Loading preprocessed dataset...")
df = pd.read_csv("network_data.csv")  # Ensure correct file path
print(f"✅ Dataset loaded! Shape: {df.shape}")

📥 Loading preprocessed dataset...
✅ Dataset loaded! Shape: (2435692, 122)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE

print("✅ Step 1: Libraries imported successfully!")

# ✅ Step 2: Extract Features and Target
features = [
    "duration", "fwd_packets_count", "bwd_packets_count",
    "fwd_total_payload_bytes", "bwd_total_payload_bytes",
    "payload_bytes_max", "payload_bytes_min",
    "fwd_payload_bytes_max", "fwd_payload_bytes_min",
    "bytes_rate", "packets_rate",
    "fwd_total_header_bytes", "bwd_total_header_bytes"
]
target = "label"

print("✅ Step 2: Features and target selected.")

X = df[features].copy()
y = df[target]

# ✅ Step 3: Handle Missing Values
X.fillna(X.mean(), inplace=True)
print("✅ Step 3: Missing values handled.")

# ✅ Step 4: Convert Target to Numeric Labels
le = LabelEncoder()
y = le.fit_transform(y)
print("✅ Step 4: Target labels encoded.")

# ✅ Step 5: Normalize Features (Critical for Deep Learning)
scaler = StandardScaler()
X = scaler.fit_transform(X)
print("✅ Step 5: Features normalized.")

# ✅ Step 6: Split Data into Train & Test Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"✅ Step 6: Data split completed - Train: {X_train.shape}, Test: {X_test.shape}")

# ✅ Step 7: Handle Class Imbalance with SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
print("✅ Step 7: SMOTE applied to balance classes.")

# ✅ Step 8: Build Optimized Model
print("🚀 Step 8: Building model...")
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),

    keras.layers.Dense(256, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(128, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(64, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(len(np.unique(y)), activation='softmax')
])
print("✅ Step 8: Model built successfully!")

# ✅ Step 9: Compile Model with Learning Rate Scheduling
lr_schedule = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("✅ Step 9: Model compiled.")

# ✅ Step 10: Train Model with More Epochs
print("🚀 Step 10: Training model...")
history = model.fit(
    X_train, y_train, epochs=20, batch_size=32,
    validation_data=(X_test, y_test), callbacks=[lr_schedule]
)
print("✅ Step 10: Model training completed!")

# ✅ Step 11: Evaluate Model
print("🚀 Step 11: Evaluating model...")
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"✅ Step 11: Test Accuracy: {test_acc:.4f}")

# ✅ Step 12: Save Model & Preprocessing Steps
model.save("network_packet_classifier_v2.h5")
print("✅ Step 12: Model saved as 'network_packet_classifier_v2.h5'")

print("🎉 Training complete! 🚀")
